In [ ]:
import tensorflow as tf
import numpy as np
import shutil
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
 
# Каталог с набором данных
data_dir = 'C:/Users/Mi/Desktop/Machine learning/train_cats_vs_dogs/train/'
# Каталог с данными для обучения
train_dir = 'C:/Users/Mi/Desktop/Machine learning/train_cats_vs_dogs/traini/'
# Каталог с данными для проверки
val_dir = 'C:/Users/Mi/Desktop/Machine learning/train_cats_vs_dogs/val/'
# Каталог с данными для тестирования
test_dir = 'C:/Users/Mi/Desktop/Machine learning/train_cats_vs_dogs/test/'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 12500
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 10
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 1750
# Количество изображений для проверки
nb_validation_samples = 375
# Количество изображений для тестирования
nb_test_samples = 375
# Архитектура модели нейронной сети
import time
# Время работы блока алгоритма
class Profiler(object):
    def __enter__(self):
        self._startTime = time.time()
         
    def __exit__(self, type, value, traceback):
        print("Elapsed time: {:.3f} sec".format(time.time() - self._startTime))

# Сверточная нейронная сеть
with Profiler() as p:
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
 
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size)
# Проверка модели на тестовых данных
    scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
    print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

# Полносвязная нейронная сеть
with Profiler() as p:
    model1 = Sequential()
    model1.add(Flatten(input_shape=input_shape))
    model1.add(Dense(32))
    model1.add(Activation('relu')) 
    model1.add(Dense(32))
    model1.add(Activation('relu')) 
    model1.add(Dense(64))
    model1.add(Activation('relu')) 
    model1.add(Dense(64))
    model1.add(Activation('relu')) 
    model1.add(Dropout(0.5))
    model1.add(Dense(1))
    model1.add(Activation('sigmoid'))
    model1.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    datagen = ImageDataGenerator(rescale=1. / 255)
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    test_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')
    model1.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size)
    scores = model1.evaluate_generator(test_generator, nb_test_samples // batch_size)
    print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))
# Тестирование модели на картинке пользователя
classes=['dog','cat']
img = load_img('z.jpg', target_size=(150, 150))
# Преобразование картинки в вектор
x =img_to_array(img)
x = x.reshape(1,150,150,3).astype('float')
x /= 255
# Применение к вектору сверточной нейронной сети
with Profiler() as p:
    pred = model.predict(x)
    pred = np.around(pred)
    print(classes[int(pred[0][0])])
# Применение к вектору полносвязной нейронной сети
with Profiler() as p:
    pred1=model1.predict(x)
    pred1 = np.around(pred1)
    print(classes[int(pred1[0][0])])


Found 15996 images belonging to 2 classes.
Found 5248 images belonging to 2 classes.
Found 3748 images belonging to 2 classes.
Epoch 1/10
1000/1000 [==============================] - 162s 162ms/step - loss: 0.6364 - acc: 0.6269 - val_loss: 0.5503 - val_acc: 0.7304
Epoch 2/10
1000/1000 [==============================] - 147s 147ms/step - loss: 0.5635 - acc: 0.7118 - val_loss: 0.5063 - val_acc: 0.7508
Epoch 3/10
1000/1000 [==============================] - 156s 156ms/step - loss: 0.5056 - acc: 0.7525 - val_loss: 0.4981 - val_acc: 0.7532
Epoch 4/10
1000/1000 [==============================] - 156s 156ms/step - loss: 0.4583 - acc: 0.7910 - val_loss: 0.4552 - val_acc: 0.7883
Epoch 5/10
1000/1000 [==============================] - 146s 146ms/step - loss: 0.4027 - acc: 0.8238 - val_loss: 0.4067 - val_acc: 0.8157
Epoch 6/10
1000/1000 [==============================] - 165s 165ms/step - loss: 0.3530 - acc: 0.8473 - val_loss: 0.4110 - val_acc: 0.8171
Epoch 7/10
1000/1000 [=======================